In [1]:
%cd ml-class/examples/lstm/time-series

/home/mb706/ml-class/examples/lstm/time-series


In [3]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [4]:
import numpy as np
import pandas as pd
import wandb
import tensorflow as tf
from plotutil import PlotCallback

wandb.init()
config = wandb.config
config.repeated_predictions = False
config.batch_size = 40
config.look_back = 4
config.epochs = 500

E1029 18:28:37.260426 140634121922368 jupyter.py:104] Failed to query for notebook name, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable
wandb: Wandb version 0.8.13 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [5]:
def load_data(data_type="airline"):
    """read a CSV into a dataframe"""
    if data_type == "flu":
        df = pd.read_csv('flusearches.csv')
        data = df.flu.astype('float32').values
    elif data_type == "airline":
        df = pd.read_csv('international-airline-passengers.csv')
        data = df.passengers.astype('float32').values
    elif data_type == "sin":
        df = pd.read_csv('sin.csv')
        data = df.sin.astype('float32').values
    return data


def create_dataset(dataset):
    """convert an array of values into a dataset matrix"""
    dataX, dataY = [], []
    for i in range(len(dataset)-config.look_back-1):
        a = dataset[i:(i+config.look_back)]
        dataX.append(a)
        dataY.append(dataset[i + config.look_back])
    return np.array(dataX), np.array(dataY)

In [8]:
data = load_data("airline")

# normalize data to between 0 and 1
max_val = max(data)
min_val = min(data)
data = (data-min_val)/(max_val-min_val)

# split into train and test sets
split = int(len(data) * 0.70)
train = data[:split]
test = data[split-config.look_back-2:]

trainX, trainY = create_dataset(train)
testX, testY = create_dataset(test)

# Add channel dimension
trainX = trainX[:, :, np.newaxis]
testX = testX[:, :, np.newaxis]

In [9]:
# create and fit the RNN
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.SimpleRNN(5, input_shape=(config.look_back, 1)))
model.add(tf.keras.layers.Dense(1, activation="sigmoid"))
model.compile(loss='mae', optimizer='rmsprop')
model.fit(trainX, trainY, epochs=config.epochs, batch_size=config.batch_size, validation_data=(testX, testY),  callbacks=[
          PlotCallback(trainX, trainY, testX, testY,
                       config.look_back, config.repeated_predictions),
          wandb.keras.WandbCallback(input_type="time")])

Train on 95 samples, validate on 45 samples
Epoch 1/500


E1029 18:34:27.804412 140634121922368 jupyter.py:104] Failed to query for notebook name, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable
wandb: Wandb version 0.8.13 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


95/95 [==============================] - 1s 14ms/sample - loss: 0.2786 - val_loss: 0.1401
Epoch 2/500
95/95 [==============================] - 0s 680us/sample - loss: 0.2664 - val_loss: 0.1476
Epoch 3/500
95/95 [==============================] - 0s 327us/sample - loss: 0.2577 - val_loss: 0.1547
Epoch 4/500
95/95 [==============================] - 0s 822us/sample - loss: 0.2506 - val_loss: 0.1630
Epoch 5/500
95/95 [==============================] - 0s 807us/sample - loss: 0.2436 - val_loss: 0.1709
Epoch 6/500
95/95 [==============================] - 0s 336us/sample - loss: 0.2369 - val_loss: 0.1795
Epoch 7/500
95/95 [==============================] - 0s 793us/sample - loss: 0.2304 - val_loss: 0.1876
Epoch 8/500
95/95 [==============================] - 0s 368us/sample - loss: 0.2240 - val_loss: 0.1961
Epoch 9/500
95/95 [==============================] - 0s 738us/sample - loss: 0.2174 - val_loss: 0.2042
Epoch 10/500
95/95 [==============================] - 0s 2ms/sample - loss: 0.2115 - v